# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** My CNN architecture uses the resnet50 architecture as recommended with a trained linear layer with batch normisation at the end. Batch normalisation was added as it showed to be helpful in the previous (facial keypoint) detection tasks. The RNN is a LSTM with 1 hidden layer and an embedding of 512 as [this paper](https://arxiv.org/pdf/1502.03044.pdf) showed to be successful with such parameters

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The transform used the same transform provided as it looked to be robust and provided the nessassary changes to ensure the images are different. Also from the previous task a similar transform was performed

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** Trainable paramters were selected based on trial and error. Initially an embedding size of 256 was used but the results didn't seem to reduce loss after some time and produced average results. Also 1 hidden layer for the RNN was chosen as it was recommended in the paper highlighted previously.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Originally the Adam Optimizer was chosen to train the model but it seemed to stop learning after a number of steps and the loss didn't decrease when reaching this threshold. A SGD was then chosen to hopefully solve this problem and find global optima instead of local optima

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

def to_var(x, volatile=False):
    """ converts a Pytorch Tensor to a variable and moves to GPU if CUDA is available """
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 10           # batch size
vocab_threshold = 5      # minimum word count threshold
vocab_from_file = True      # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.SGD(params, lr=0.1)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.12s)
creating index...


  0%|          | 545/414113 [00:00<01:15, 5444.02it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:55<00:00, 7429.10it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
# response = requests.request("GET", 
#                             "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                             headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
#         if time.time() - old_time > 60:
#             old_time = time.time()
#             requests.request("POST", 
#                              "https://nebula.udacity.com/api/v1/remote/keep-alive", 
#                              headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        for batch in data_loader:
            images, captions = batch[0], batch[1]
            break 
        
        # Convert batch of images and captions to Pytorch Variable.
        images = to_var(images, volatile=True)
        captions = to_var(captions)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.data[0], np.exp(loss.data[0]))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('models', 'decoder-v4-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('models', 'encoder-v4-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [1/41412], Loss: 9.0889, Perplexity: 8856.2324

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/3], Step [100/41412], Loss: 6.6478, Perplexity: 771.0596
Epoch [1/3], Step [200/41412], Loss: 5.7420, Perplexity: 311.70079
Epoch [1/3], Step [300/41412], Loss: 4.9804, Perplexity: 145.5257
Epoch [1/3], Step [400/41412], Loss: 5.0365, Perplexity: 153.9371
Epoch [1/3], Step [500/41412], Loss: 4.7532, Perplexity: 115.9495
Epoch [1/3], Step [600/41412], Loss: 3.5847, Perplexity: 36.04310
Epoch [1/3], Step [700/41412], Loss: 3.6731, Perplexity: 39.37252
Epoch [1/3], Step [800/41412], Loss: 3.6067, Perplexity: 36.84258
Epoch [1/3], Step [900/41412], Loss: 3.7803, Perplexity: 43.82751
Epoch [1/3], Step [1000/41412], Loss: 4.4711, Perplexity: 87.4491
Epoch [1/3], Step [1100/41412], Loss: 3.9566, Perplexity: 52.28049
Epoch [1/3], Step [1200/41412], Loss: 4.6324, Perplexity: 102.7625
Epoch [1/3], Step [1300/41412], Loss: 4.6677, Perplexity: 106.4546
Epoch [1/3], Step [1400/41412], Loss: 3.9046, Perplexity: 49.62948
Epoch [1/3], Step [1500/41412], Loss: 3.2681, Perplexity: 26.26080
Epoc

Epoch [1/3], Step [15600/41412], Loss: 2.8493, Perplexity: 17.2751
Epoch [1/3], Step [15700/41412], Loss: 2.0323, Perplexity: 7.63170
Epoch [1/3], Step [15800/41412], Loss: 2.5894, Perplexity: 13.3212
Epoch [1/3], Step [15900/41412], Loss: 2.6790, Perplexity: 14.5703
Epoch [1/3], Step [16000/41412], Loss: 2.5273, Perplexity: 12.5195
Epoch [1/3], Step [16100/41412], Loss: 2.5746, Perplexity: 13.1265
Epoch [1/3], Step [16200/41412], Loss: 2.5078, Perplexity: 12.2782
Epoch [1/3], Step [16300/41412], Loss: 2.2623, Perplexity: 9.60507
Epoch [1/3], Step [16400/41412], Loss: 2.8572, Perplexity: 17.4131
Epoch [1/3], Step [16500/41412], Loss: 2.8820, Perplexity: 17.8501
Epoch [1/3], Step [16600/41412], Loss: 2.3583, Perplexity: 10.5725
Epoch [1/3], Step [16700/41412], Loss: 2.7888, Perplexity: 16.2614
Epoch [1/3], Step [16800/41412], Loss: 3.3817, Perplexity: 29.4198
Epoch [1/3], Step [16900/41412], Loss: 2.8491, Perplexity: 17.2714
Epoch [1/3], Step [17000/41412], Loss: 2.4153, Perplexity: 11.

Epoch [1/3], Step [27800/41412], Loss: 2.7159, Perplexity: 15.1177
Epoch [1/3], Step [27900/41412], Loss: 2.9484, Perplexity: 19.0757
Epoch [1/3], Step [28000/41412], Loss: 2.3872, Perplexity: 10.8833
Epoch [1/3], Step [28100/41412], Loss: 2.8128, Perplexity: 16.6569
Epoch [1/3], Step [28200/41412], Loss: 2.1302, Perplexity: 8.41670
Epoch [1/3], Step [28300/41412], Loss: 2.0719, Perplexity: 7.93991
Epoch [1/3], Step [28400/41412], Loss: 2.5823, Perplexity: 13.2272
Epoch [1/3], Step [28500/41412], Loss: 2.7802, Perplexity: 16.1219
Epoch [1/3], Step [28600/41412], Loss: 3.1719, Perplexity: 23.8526
Epoch [1/3], Step [28700/41412], Loss: 2.5481, Perplexity: 12.7830
Epoch [1/3], Step [28800/41412], Loss: 2.4110, Perplexity: 11.1447
Epoch [1/3], Step [28900/41412], Loss: 1.8679, Perplexity: 6.47449
Epoch [1/3], Step [29000/41412], Loss: 3.1380, Perplexity: 23.0583
Epoch [1/3], Step [29100/41412], Loss: 2.4752, Perplexity: 11.8836
Epoch [1/3], Step [29200/41412], Loss: 2.9181, Perplexity: 18.

...

Epoch [2/3], Step [23100/41412], Loss: 2.4266, Perplexity: 11.3203
Epoch [2/3], Step [23200/41412], Loss: 2.4887, Perplexity: 12.0459
Epoch [2/3], Step [23300/41412], Loss: 2.0618, Perplexity: 7.85999
Epoch [2/3], Step [23400/41412], Loss: 3.1612, Perplexity: 23.5979
Epoch [2/3], Step [23500/41412], Loss: 2.2746, Perplexity: 9.72416
Epoch [2/3], Step [23600/41412], Loss: 2.6689, Perplexity: 14.4235
Epoch [2/3], Step [23700/41412], Loss: 2.4531, Perplexity: 11.6242
Epoch [2/3], Step [23800/41412], Loss: 2.1933, Perplexity: 8.96481
Epoch [2/3], Step [23900/41412], Loss: 2.4230, Perplexity: 11.2796
Epoch [2/3], Step [24000/41412], Loss: 2.6632, Perplexity: 14.3419
Epoch [2/3], Step [24100/41412], Loss: 2.0812, Perplexity: 8.01451
Epoch [2/3], Step [24200/41412], Loss: 2.6499, Perplexity: 14.1528
Epoch [2/3], Step [24300/41412], Loss: 3.0858, Perplexity: 21.8859
Epoch [2/3], Step [24400/41412], Loss: 2.2744, Perplexity: 9.72234
Epoch [2/3], Step [24500/41412], Loss: 2.4573, Perplexity: 11.

Epoch [2/3], Step [35300/41412], Loss: 2.2821, Perplexity: 9.79748
Epoch [2/3], Step [35400/41412], Loss: 2.6984, Perplexity: 14.8554
Epoch [2/3], Step [35500/41412], Loss: 1.9561, Perplexity: 7.07205
Epoch [2/3], Step [35600/41412], Loss: 2.2313, Perplexity: 9.31161
Epoch [2/3], Step [35700/41412], Loss: 2.0698, Perplexity: 7.92357
Epoch [2/3], Step [35800/41412], Loss: 1.9471, Perplexity: 7.00841
Epoch [2/3], Step [35900/41412], Loss: 2.1737, Perplexity: 8.79086
Epoch [2/3], Step [36000/41412], Loss: 2.6135, Perplexity: 13.6462
Epoch [2/3], Step [36100/41412], Loss: 2.3231, Perplexity: 10.2076
Epoch [2/3], Step [36200/41412], Loss: 1.7739, Perplexity: 5.894105
Epoch [2/3], Step [36300/41412], Loss: 2.2673, Perplexity: 9.65379
Epoch [2/3], Step [36400/41412], Loss: 2.2025, Perplexity: 9.04727
Epoch [2/3], Step [36500/41412], Loss: 2.2696, Perplexity: 9.67518
Epoch [2/3], Step [36600/41412], Loss: 2.5252, Perplexity: 12.4931
Epoch [2/3], Step [36700/41412], Loss: 2.5864, Perplexity: 13

Epoch [3/3], Step [6200/41412], Loss: 2.2666, Perplexity: 9.64647
Epoch [3/3], Step [6300/41412], Loss: 2.0893, Perplexity: 8.07892
Epoch [3/3], Step [6400/41412], Loss: 2.2714, Perplexity: 9.69266
Epoch [3/3], Step [6500/41412], Loss: 2.0085, Perplexity: 7.45255
Epoch [3/3], Step [6600/41412], Loss: 2.5243, Perplexity: 12.4823
Epoch [3/3], Step [6700/41412], Loss: 2.0924, Perplexity: 8.10470
Epoch [3/3], Step [6800/41412], Loss: 2.2069, Perplexity: 9.08746
Epoch [3/3], Step [6900/41412], Loss: 2.0081, Perplexity: 7.44899
Epoch [3/3], Step [7000/41412], Loss: 2.7843, Perplexity: 16.1883
Epoch [3/3], Step [7100/41412], Loss: 2.5758, Perplexity: 13.1419
Epoch [3/3], Step [7200/41412], Loss: 2.1599, Perplexity: 8.67067
Epoch [3/3], Step [7300/41412], Loss: 2.4821, Perplexity: 11.9664
Epoch [3/3], Step [7400/41412], Loss: 2.6447, Perplexity: 14.0795
Epoch [3/3], Step [7500/41412], Loss: 2.2258, Perplexity: 9.26095
Epoch [3/3], Step [7600/41412], Loss: 1.8976, Perplexity: 6.67001
Epoch [3/3

Epoch [3/3], Step [18400/41412], Loss: 2.6378, Perplexity: 13.9818
Epoch [3/3], Step [18500/41412], Loss: 1.6457, Perplexity: 5.18460
Epoch [3/3], Step [18600/41412], Loss: 2.7285, Perplexity: 15.3104
Epoch [3/3], Step [18700/41412], Loss: 2.3590, Perplexity: 10.5801
Epoch [3/3], Step [18800/41412], Loss: 2.9451, Perplexity: 19.0127
Epoch [3/3], Step [18900/41412], Loss: 1.9010, Perplexity: 6.69272
Epoch [3/3], Step [19000/41412], Loss: 2.1076, Perplexity: 8.22859
Epoch [3/3], Step [19100/41412], Loss: 2.9943, Perplexity: 19.9707
Epoch [3/3], Step [19200/41412], Loss: 2.3419, Perplexity: 10.4013
Epoch [3/3], Step [19300/41412], Loss: 2.8684, Perplexity: 17.6087
Epoch [3/3], Step [19400/41412], Loss: 2.4888, Perplexity: 12.0463
Epoch [3/3], Step [19500/41412], Loss: 2.0558, Perplexity: 7.81284
Epoch [3/3], Step [19600/41412], Loss: 2.0246, Perplexity: 7.57291
Epoch [3/3], Step [19700/41412], Loss: 2.0000, Perplexity: 7.38912
Epoch [3/3], Step [19800/41412], Loss: 2.3213, Perplexity: 10.

Epoch [3/3], Step [30600/41412], Loss: 2.0966, Perplexity: 8.13814
Epoch [3/3], Step [30700/41412], Loss: 1.8874, Perplexity: 6.60198
Epoch [3/3], Step [30800/41412], Loss: 2.0875, Perplexity: 8.06518
Epoch [3/3], Step [30900/41412], Loss: 1.7863, Perplexity: 5.96719
Epoch [3/3], Step [31000/41412], Loss: 1.9493, Perplexity: 7.02374
Epoch [3/3], Step [31100/41412], Loss: 2.3743, Perplexity: 10.7434
Epoch [3/3], Step [31200/41412], Loss: 2.0026, Perplexity: 7.40849
Epoch [3/3], Step [31300/41412], Loss: 2.1069, Perplexity: 8.22250
Epoch [3/3], Step [31400/41412], Loss: 2.3899, Perplexity: 10.9129
Epoch [3/3], Step [31500/41412], Loss: 2.4160, Perplexity: 11.2015
Epoch [3/3], Step [31600/41412], Loss: 2.1029, Perplexity: 8.19000
Epoch [3/3], Step [31700/41412], Loss: 2.2315, Perplexity: 9.31354
Epoch [3/3], Step [31800/41412], Loss: 1.9681, Perplexity: 7.15724
Epoch [3/3], Step [31900/41412], Loss: 2.8497, Perplexity: 17.2823
Epoch [3/3], Step [32000/41412], Loss: 2.3242, Perplexity: 10.

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.